# Data exploration

In [1]:
# import jsonlines

# count = 0

# with jsonlines.open('Data/07-13-2020.ud.jsonl') as f:
#     for line in f.iter():
#         count += 1
        
# print(count)

In [2]:
import pandas as pd    
raw_data = pd.read_json(path_or_buf='Data/07-13-2020.ud.jsonl', lines=True)
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3857168 entries, 0 to 3857167
Data columns (total 11 columns):
 #   Column        Dtype         
---  ------        -----         
 0   defid         int64         
 1   row           int64         
 2   word          object        
 3   meaning       object        
 4   example       object        
 5   contributor   object        
 6   date          datetime64[ns]
 7   upvotes       int64         
 8   downvotes     int64         
 9   crawled_time  datetime64[ns]
 10  gif           object        
dtypes: datetime64[ns](2), int64(4), object(5)
memory usage: 323.7+ MB


File contain 3857168 definitions

Each row contains a definition for a word -> there can be multiple definitions for a word, and mutliple capitalisations of a word too

In [3]:
# Age range of data
print(raw_data.iloc[raw_data["date"].argmax()]) # 2020-07-13
print(raw_data.iloc[raw_data["date"].argmin()]) # 1999-12-09

defid                                                    15176700
row                                                             3
word                                        Comically Large Spoon
meaning         A spoon that was present in King Bach's ice cr...
example         "Can I get some ice cream?""Only a spoonful."\...
contributor                                                 Bukas
date                                          2020-07-13 00:00:00
upvotes                                                         0
downvotes                                                       0
crawled_time                                  2020-07-13 15:53:13
gif             https://media3.giphy.com/media/5CxttXrbRlaCs/g...
Name: 1432193, dtype: object
defid                                                          12
row                                                            10
word                                                    hard-core
meaning         anything out of our league that

In [4]:
# Dump rows we don't need
del raw_data['row']
del raw_data['contributor']
del raw_data['date']
del raw_data['crawled_time']
del raw_data['gif']

# make all words lower case for ease of computation
raw_data['word'] = raw_data['word'].str.lower()

print(raw_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3857168 entries, 0 to 3857167
Data columns (total 6 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   defid      int64 
 1   word       object
 2   meaning    object
 3   example    object
 4   upvotes    int64 
 5   downvotes  int64 
dtypes: int64(3), object(3)
memory usage: 176.6+ MB
None


In [5]:
def get_slurs(include, exclude, duplicates = False, df = raw_data):
    # include - list of terms use during search of data
    # exclude - list of terms to exclude from words found by include search
    # return - unique terms that contain include terms and does not contain exclude terms
    if not exclude:
        tmp = df[raw_data.meaning.str.contains('|'.join(include))]
    else:
        
        tmp = df[df.meaning.str.contains('|'.join(include))]
        tmp = tmp[~tmp.meaning.str.contains('|'.join(exclude))]
    
    if duplicates:
        return tmp
    else:
        return tmp.drop_duplicates(subset='word')

Keyphrases:
- derogatory term, insulting term, racist term, reclaimed, prejudice, slur
- examples are questionable?

questionable:
- ?

If it contains certain terms -> include it

Do we want to use a ranking system to select terms as some descriptions may have

Do we want to consider contreversial terms which are potentially still used? Could use in the filtering.

In [21]:
include_terms = ['derogatory', 'insulting', 'racist', 'prejudice', ' slur ']
exclude_terms = ['website', ' shutting down racist ', 'event']

slurs = get_slurs(include_terms, exclude_terms)

print(slurs)

            defid            word  \
15       10000237        keemstar   
186       1000172     abercrombie   
341      10003297       trumpence   
635      10005969           locky   
1182     10011611   speckled coon   
...           ...             ...   
3856295   9991558         raceism   
3856353   9992118  chinese wanker   
3856426    999302          cetnik   
3856602   9994685      rash trump   
3856775   9996067            soup   

                                                   meaning  \
15       A racist garden gnome who loves youtube revenu...   
186      1. a clothing store that is highly overpriced,...   
341      The act of insincerely apologizing after makin...   
635      Locky is a derogatory word that is the male fo...   
1182     A type of dog. Definetley not a derogatory nam...   
...                                                    ...   
3856295  noun: raceismthe belief that all members of ea...   
3856353  1. A wanker of the Chinese variety. Usually at... 

In [8]:
# refined = slurs

# keyphrases = [' less racist atmosphere ']

# matches = refined.meaning.str.contains('|'.join(keyphrases))

# slurs = refined[matches].drop_duplicates(subset='word')

# keyphrases = [' website ']

# matches = refined.meaning.str.contains('|'.join(keyphrases))

# refined = refined[matches].drop_duplicates(subset='word')

uses of racist:
- closeted racist who
- a racist gnome
- outright racist remarks
- as racist -> has some issues with certain terms, but lots of good uses 
- a racist stance
- less racist atmosphere/in -> remove?
- less racist way/version/ - keep

- event -> get rid of?

thoughts:
- Would something that is originally a name be considered a slur - i.e. Keemstar
- Could use NER to detect events, real names(A lot of them are slurs) and remove them? Training set not possible

Could be interesting to track the use of names/events and their use? Is it worth it?